
In this example, we build and train a deep learning model using the FashionMNIST dataset, which consists of 28x28 grayscale images of 10 different clothing items. The model, a convolutional neural network (CNN), is trained to classify these images into their respective categories, leveraging PyTorch for the implementation and employing techniques such as normalization and Adam optimization to enhance performance.

## Steps to be followed:

1. Import necessary libraries
2. Data Preprocessing Setup
3. Load Datasets
4. Initialize DataLoaders
5. Define the CNN Model
6. Set Up Loss Function and Optimizer
7. Training the Model
8. Evaluate the Model


### Step 1: Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

### Step 2:  Data Preprocessing Setup
- Define transformations for the input data:
     - `ToTensor():` Converts image data from PIL format or NumPy arrays to PyTorch tensors.
     - `Normalize((0.5,), (0.5,)):` Normalizes tensor images using mean = 0.5 and std = 0.5.

In [2]:
# Data preprocessing: normalization
transform_nm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

### Step 3: Load Datasets
- **Train Dataset:** Load FashionMNIST training data, applying the defined transformations.
- **Test Dataset:** Load FashionMNIST test data similarly.

In [3]:
# Loading the dataset
train_dataset_nm = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform_nm)
test_dataset_nm = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform_nm)

100%|██████████| 26421880/26421880 [00:01<00:00, 18498656.01it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 338885.53it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6090756.96it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3774213.77it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



### Step 4: Initialize DataLoaders
- `Training DataLoader:` Batches, shuffles, and prepares training data for processing in the model.
- `Testing DataLoader:` Batches and prepares test data for evaluation (shuffling is not necessary for testing)

In [4]:
train_loader_nm = torch.utils.data.DataLoader(train_dataset_nm, batch_size=32, shuffle=True)
test_loader_nm = torch.utils.data.DataLoader(test_dataset_nm, batch_size=32, shuffle=False)

# Printing the shape of the datasets
print(f'Training data: {len(train_dataset_nm)} samples')
print(f'Testing data: {len(test_dataset_nm)} samples')

Training data: 60000 samples
Testing data: 10000 samples


### Step 5: Define the CNN Model
- **Model Class Initialization:** Set up the neural network structure with layers defined in the `__init__` method.
- **Layers:** Include one convolutional layer, one pooling layer, and two fully connected layers.
- **Data Flow through Layers:** Define how data moves through the model using the forward method, incorporating activations (ReLU) and pooling

In [5]:
# Define a convolutional neural network for classifying FashionMNIST dataset images
class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super(FashionMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc_hidden = nn.Linear(13*13*32, 100)
        self.fc_output = nn.Linear(100, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 13*13*32)
        x = self.fc_hidden(x)
        x = F.relu(x)
        x = self.fc_output(x)
        return x



### Step 6: Set Up Loss Function and Optimizer

- **CrossEntropyLoss:** Used for multi-class classification tasks.
- **Adam Optimizer:** A method for stochastic optimization with a set learning rate of 0.001.

In [6]:
model = FashionMNISTCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Step 7: Training the Model and saving it at each epoch

- **Epoch Iteration:** Loop through the dataset multiple times (epochs).
- **Batch Processing:** For each batch in the DataLoader, perform forward pass, loss calculation, backpropagation, and parameter update.
- **Save Model State:** Save the model after training to reuse it later without needing to retrain.

In [7]:
# Training the model for 10 epochs
# Define the path for saving the model
model_path = './mnist_model.pth'

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images_nm, labels_nm in train_loader_nm:
        optimizer.zero_grad()

        outputs_nm = model(images_nm)
        loss = criterion(outputs_nm, labels_nm)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted_train = torch.max(outputs_nm.data, 1)
        total_train += labels_nm.size(0)
        correct_train += (predicted_train == labels_nm).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader_nm)}, Train Accuracy: {train_accuracy}%")

    # Save the model at the end of each epoch
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

Epoch 1/10, Loss: 0.39879183058540024, Train Accuracy: 85.74666666666667%
Model saved to ./mnist_model.pth
Epoch 2/10, Loss: 0.26612817190289495, Train Accuracy: 90.37333333333333%
Model saved to ./mnist_model.pth
Epoch 3/10, Loss: 0.2216284805228313, Train Accuracy: 91.945%
Model saved to ./mnist_model.pth
Epoch 4/10, Loss: 0.1883942069063584, Train Accuracy: 93.05%
Model saved to ./mnist_model.pth
Epoch 5/10, Loss: 0.1620183053433895, Train Accuracy: 94.01166666666667%
Model saved to ./mnist_model.pth
Epoch 6/10, Loss: 0.13680088447729746, Train Accuracy: 94.945%
Model saved to ./mnist_model.pth
Epoch 7/10, Loss: 0.1175761468090117, Train Accuracy: 95.71%
Model saved to ./mnist_model.pth
Epoch 8/10, Loss: 0.09986156567347547, Train Accuracy: 96.26666666666667%
Model saved to ./mnist_model.pth
Epoch 9/10, Loss: 0.08623724563966195, Train Accuracy: 96.845%
Model saved to ./mnist_model.pth
Epoch 10/10, Loss: 0.074453572653234, Train Accuracy: 97.26166666666667%
Model saved to ./mnist_mo

### Step 8:  Evaluate the Model
- **Switch to Evaluation Mode:** Ensure the model is in eval mode to disable dropout or batch norm effects during testing.
- **Accuracy Calculation:** Compare the model’s output to the true labels, calculate overall accuracy.

In [8]:
# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images_nm, labels_nm in test_loader_nm:
        outputs_nm = model(images_nm)
        _, predicted = torch.max(outputs_nm.data, 1)
        total += labels_nm.size(0)
        correct += (predicted == labels_nm).sum().item()

accuracy = 100 * correct / total
print(f"Model accuracy on test set: {accuracy}%")

Model accuracy on test set: 91.45%
